In [24]:
# Variable
W_max = 19.5;
W_min = 10;
import xlrd
import xlwt
import os


## excel parameter
# parameter schedule end Index
Schedule_endIndex = 289
# student schedule end Index
EndTime = 39;

# function for formating
def timeFormating(num):
    n = str(num)
    time = n.split(".")
    if time[1]=='5':
        time[1]='50' 
    min=str(int(time[1])*60/100).split(".")[0]
    if min =='0':
        min='00'
    if int(time[0]) <12:
        return time[0]+":"+min+"am"
    elif int(time[0])==12:
        return time[0]+":"+min+"pm"
    elif int(time[0]) >12:
        return str(int(time[0])-12)+":"+min+"pm"
    
def dayFormating(num):
    if num==1:
        return "Mon"
    elif num ==2:
        return "Tue"
    elif num==3:
        return "Wed"
    elif num==4:
        return "Thu"
    elif num==5:
        return "Fri"
    elif num==6:
        return "Sat"
    elif num==7:
        return "Sun"

loc = ("schedule.xls")
wb = xlrd.open_workbook(loc)
sheet = wb.sheet_by_index(0)
x = list(range(1, Schedule_endIndex))
#initial data
startTime = []
processStartTime = []
endTime = []
shiftslength = []
days = []
shiftstations = []
station = []

# get shift data
for i in x:
    timeStarted = sheet.cell_value(i,1)
    startTime.append(timeStarted)
    if str(timeStarted)[-3:] ==".25" or str(timeStarted)[-3:] ==".75" :
        processStartTime.append(timeStarted-0.25)
    else :
        processStartTime.append(timeStarted)  
    endTime.append(sheet.cell_value(i,2))
    shiftslength.append(sheet.cell_value(i,3))
    days.append(sheet.cell_value(i,4))
    shiftstations.append(sheet.cell_value(i,5))
    if sheet.cell_value(i,5) not in station:
        station.append(sheet.cell_value(i,5))

shifts = list(range(0, Schedule_endIndex-1))
# get all students availability and prefer time
studentName = []
p = []
#allAvailability[i]:availability for student i
#allAvailability[i][j]:availability for student i on day j
#allAvailability[i][j][k]:availability for student i on day j at time k
allAvailability = []
dirs = os.listdir("studentAvailability")
for file in dirs:
    if (not file.startswith("~$")) and (not file.startswith(".")) and (not file.startswith("_")):
        studentName.append(file.split(".")[0])
        # get each student availability
        wbp=xlrd.open_workbook("studentAvailability/"+file)
        sheetperson = wbp.sheet_by_index(0)
        # get prefer working time
        p.append(sheetperson.cell_value(1,8))
        time_slot = list(range(1, EndTime))
        stuAvailability = []
        for j in list(range(1,8)):
            day_slot = []
            for i in time_slot:
                day_slot.append(sheetperson.cell_value(i,j))
            stuAvailability.append(day_slot)
        allAvailability.append(stuAvailability)
# get whether shift is available for each student
a=[]
for i in range(len(studentName)):
    stuShiftAvailability = []
    stuAvailability = allAvailability[i]
    #loop all shifts
    for j in shifts:
        shiftstartTime = startTime[j]
        shiftLength = shiftslength[j]
        day = int(days[j]-1)
        startTimeSlot = int((shiftstartTime-7)/0.5)
        lengthSlot = int(shiftLength/0.5)
        # get student availability for the specific day
        stuDayAvailability = stuAvailability[day]
        # loop one single shift
        available = 1;
        for k in list(range(startTimeSlot,startTimeSlot+lengthSlot)):
            if(stuDayAvailability[k] ==0):
                available=0
                break
        stuShiftAvailability.append(available)
    a.append(stuShiftAvailability)

notavailable = []
for i in shifts:
    for j in range(i, len(shifts)):
        if days[i] == days[j] and i !=j:
            if startTime[i] <= startTime[j] and endTime[i] > startTime[j]:
                notavailable.append([i,j])
            elif endTime[j] > startTime[i] and endTime[j] <= endTime[i]:
                notavailable.append([i,j])

In [25]:
#del allAvailability
# model building
from gurobipy import *
import statistics
m = Model()
# initial variable
#x[i][j] assign student i to shift j
x=[]
for i in range(len(studentName)):
    y = [None]*len(shifts)
    x.append(y)
for i in range(len(studentName)):
    for j in shifts:
        x[i][j] = m.addVar(vtype=GRB.BINARY)
m.update()

# add constrain
for i in range(len(studentName)):
    expr = LinExpr()
    for j in shifts:
        if a[i][j] ==0:
            m.addConstr(x[i][j]==0)
        expr.add(x[i][j],shiftslength[j])
    m.addConstr(expr<=W_max)
    m.addConstr(expr>=W_min)

    for k in notavailable:
        expr2 = LinExpr()
        for g in range(len(k)):
            expr2.add(x[i][g],1)
        m.addConstr(expr2 <= 1)

for j in shifts:
    expr = LinExpr()
    for i in range(len(studentName)):
        expr.add(x[i][j],1)
    m.addConstr(expr<=1)

# goal
# F1
F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = []
    lateNightshift = 0
    for j in shifts:
        if x[i][j] == 1:
            scheduleTime += x[i][j]*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
            
            if shiftstations[j] == "Late Night":
                lateNightshift+=1
                
    F1 += (scheduleTime-p[i])*(scheduleTime-p[i])
    F2 += len(stuStation)-lateNightshift-len(unique)
# Define the objective function
ohm_1 = 2
ohm_2 = 10
m.Params.MIPGapAbs =0.00001
m.setObjective((ohm_1*F1+ ohm_2*F2)/len(studentName), GRB.MINIMIZE)
m.optimize()

Changed value of parameter MIPGapAbs to 1e-05
   Prev: 1e-10  Min: 0.0  Max: 1e+100  Default: 1e-10
Optimize a model with 122622 rows, 14400 columns and 279784 nonzeros
Model has 2080800 quadratic objective terms
Variable types: 0 continuous, 14400 integer (14400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 6e+00]
  QObjective range [2e-01, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 2668.6300000
Presolve removed 122218 rows and 7884 columns
Presolve time: 0.67s
Presolved: 404 rows, 6516 columns, 19580 nonzeros
Presolved model has 461462 quadratic objective terms
Variable types: 0 continuous, 6516 integer (6516 binary)

Root relaxation: objective 2.650000e+03, 1043 iterations, 0.10 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2650.00000    0   75 

In [30]:
workbook = xlwt.Workbook()
sheet = workbook.add_sheet('schedule')
sheet.write(0,0,"Shift")
sheet.write(0,1,"Time")
sheet.write(0,2,"Station")
for k, value in enumerate(shifts):
    sheet.write(int(k)+1, 0, k+1)
    sheet.write(int(k)+1, 1, dayFormating(days[k])+":"+timeFormating(startTime[k])+"-"+timeFormating(endTime[k]))
    sheet.write(int(k)+1, 2, shiftstations[k])
for j in range(0,len(studentName)):
    sheet.write(0,3+j,studentName[j])
    f = open("schedule/"+studentName[j]+".txt", "w")
    f.write(studentName[j]+":\n")
    writeMessage = ''
    totalHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalHour += x[j][i].x*shiftslength[i]
            sheet.write(i+1,3+j,"Assign")
            writeMessage += dayFormating(days[i])+": "+timeFormating(startTime[i])+"-"+\
                            timeFormating(endTime[i])+", ("+str(shiftstations[i])+")\n"
    f.write('Total hour: '+str(totalHour)+'\n')
    f.write(writeMessage)
    f.close()
workbook.save('assignSchedule.xls')
#check objective function 1
print("Result for objective function 1")
difference = 0
print("Student: Total time:\tPrefer:\tDifference")
for j in range(0,len(studentName)):
    writeMessage = ''
    totalstuHour = 0
    for i in shifts:
        if (type(x[j][i]) != int) and (x[j][i].x==1) :
            totalstuHour += x[j][i].x*shiftslength[i]
    difference+=abs(totalstuHour-p[j])
    print(studentName[j]+"\t "+str(totalstuHour)+"\t\t"+str(p[j])+"\t"+str(abs(totalstuHour-p[j])))
print("Total difference is "+str(difference))
print()
print()
# check objective function 2
print("Result for objective function 2")
totalStation = 0
totalunique = 0
print("Student: No. shifts:\tNo. different station:\tDifference")
for i in range(len(studentName)):
    stuStation = []
    unique = []
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    totalStation+=len(stuStation)
    totalunique+=len(unique)
    print(studentName[i]+"\t "+str(len(stuStation))+"\t\t"+str(len(unique))+"\t\t\t"+str(len(stuStation)- len(unique)))
print("Total\t "+str(totalStation)+"\t\t"+str(totalunique)+"\t\t\t"+str(totalStation-totalunique))

F1 = 0
F2 = 0
for i in range(len(studentName)):
    scheduleTime = 0
    stuStation = []
    unique = []
    for j in shifts:
        if x[i][j].x == 1:
            scheduleTime += x[i][j].x*shiftslength[j]
            stuStation.append(shiftstations[j])
            if shiftstations[j] not in unique :
                unique.append(shiftstations[j])
    F1 += (scheduleTime-p[i])*(scheduleTime-p[i])
    F2 += len(stuStation)-len(unique)
print()
print(F1)
print(F2)
ohm_1 = 2
ohm_2 = 5
print((ohm_1*F1+ ohm_2*F2)/len(studentName))

Result for objective function 1
Student: Total time:	Prefer:	Difference
11	 13.0		13.0	0.0
39	 15.5		15.5	0.0
38	 14.0		14.0	0.0
10	 15.0		15.0	0.0
12	 15.0		15.0	0.0
13	 10.0		10.0	0.0
17	 12.0		12.0	0.0
16	 14.0		14.0	0.0
28	 11.0		12.0	1.0
14	 13.5		13.5	0.0
15	 16.25		16.0	0.25
29	 15.25		15.0	0.25
8	 12.75		12.0	0.75
9	 17.0		17.0	0.0
48	 10.5		10.5	0.0
49	 13.5		13.5	0.0
1	 10.0		10.0	0.0
47	 10.0		10.0	0.0
46	 14.25		14.5	0.25
44	 11.5		11.5	0.0
50	 10.0		10.0	0.0
2	 13.75		13.5	0.25
3	 13.0		13.0	0.0
45	 12.5		12.5	0.0
41	 10.0		10.0	0.0
7	 13.5		14.0	0.5
6	 13.25		14.0	0.75
40	 13.25		13.0	0.25
4	 11.0		11.0	0.0
42	 10.75		11.0	0.25
43	 13.5		13.5	0.0
5	 12.75		12.5	0.25
30	 14.0		14.0	0.0
24	 12.0		12.0	0.0
18	 12.25		12.0	0.25
19	 13.25		12.5	0.75
25	 11.0		11.0	0.0
31	 14.0		13.5	0.5
27	 14.25		15.0	0.75
33	 13.25		13.0	0.25
32	 14.5		14.5	0.0
26	 13.5		13.5	0.0
22	 16.0		16.0	0.0
36	 12.0		12.0	0.0
37	 16.25		17.0	0.75
23	 17.75		17.0	0.75
35	 13.5		14.0	0.5
21	 14.0		14.0

In [29]:
# print remaining shifts
print("Shift\tStart Time\tEnd Time\tLength\tDay\tStation")
n = 0
for j in shifts:
    assign = 0
    for i in range(len(studentName)):
        if x[i][j].x==1:
            assign=1
            break
    if assign == 0:
        n+=1
        print(j,"\t",startTime[j],"\t\t",endTime[j],"\t\t",shiftslength[j],"\t",days[j],"\t",shiftstations[j])
print("Total number of unassigned shifts: ",n)

Shift	Start Time	End Time	Length	Day	Station
0 	 7.0 		 10.5 		 3.5 	 1.0 	 Big Kitchen
4 	 16.75 		 20.0 		 3.25 	 1.0 	 Big Kitchen
5 	 17.0 		 20.5 		 3.5 	 1.0 	 Big Kitchen
8 	 10.75 		 15.0 		 4.25 	 2.0 	 Big Kitchen
11 	 17.0 		 20.5 		 3.5 	 2.0 	 Big Kitchen
12 	 7.0 		 10.5 		 3.5 	 3.0 	 Big Kitchen
18 	 7.0 		 10.5 		 3.5 	 4.0 	 Big Kitchen
22 	 16.75 		 20.0 		 3.25 	 4.0 	 Big Kitchen
23 	 17.0 		 20.5 		 3.5 	 4.0 	 Big Kitchen
26 	 10.75 		 15.0 		 4.25 	 5.0 	 Big Kitchen
30 	 9.5 		 13.25 		 3.75 	 6.0 	 Big Kitchen
32 	 10.25 		 14.25 		 4.0 	 6.0 	 Big Kitchen
33 	 10.75 		 14.5 		 3.75 	 6.0 	 Big Kitchen
36 	 9.5 		 13.25 		 3.75 	 7.0 	 Big Kitchen
38 	 10.25 		 14.25 		 4.0 	 7.0 	 Big Kitchen
39 	 10.75 		 14.5 		 3.75 	 7.0 	 Big Kitchen
48 	 16.75 		 20.5 		 3.75 	 1.0 	 Deli
57 	 16.75 		 20.5 		 3.75 	 2.0 	 Deli
61 	 7.5 		 11.0 		 3.5 	 3.0 	 Deli
62 	 10.75 		 14.0 		 3.25 	 3.0 	 Deli
66 	 16.75 		 20.5 		 3.75 	 3.0 	 Deli
68 	 17.75 		 21.0 		 3.25 